In [9]:
import numpy as np
import pandas as pd
import tqdm
import time
import pickle

from scipy.special import comb as binom
from lib import svm_proba
from lib import enm_proba_exact
from lib import enm_proba_apprx

from concurrent.futures import ThreadPoolExecutor

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
SEED = 8098
np.random.seed(SEED)

In [3]:
def timer(func, *args, **kwargs):
    start = time.time()
    result = func(*args, **kwargs)
    end = time.time()
    return result, end - start

## Experiment

In [15]:
N_MIN = 2
N_MAX = 5 + 1
K = 0

N_THREADS = 2

In [16]:
def run(N0, N1, k):
    X0 = np.random.multivariate_normal(
        [0, 0],
        [[1, 0], [0, 1]],
        size=N0
    )
    y0 = np.zeros(N0, dtype=int) + 0

    X1 = np.random.multivariate_normal(
        [0, 0],
        [[1, 0], [0, 1]],
        size=N1
    )
    y1 = np.zeros(N1, dtype=int) + 1

    X = np.vstack([X0, X1], dtype=np.float32)
    y = np.concatenate([y0, y1], dtype=np.int32)

    result, time = timer(enm_proba_exact, X, y, k)
    nominator, denominator = result

    return nominator, denominator, time

In [17]:
futures = []
bank = []

with ThreadPoolExecutor(N_THREADS) as pool:
    for n in range(N_MIN, N_MAX + 1):
        futures.append(pool.submit(run, n, n, K))

for future in tqdm.tqdm(futures):
    bank.append(future.result())

100%|█████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 79437.58it/s]


In [18]:
print(bank)

[(6, 6, 0.0009398460388183594), (8, 20, 0.0034377574920654297), (12, 70, 0.004848957061767578), (14, 252, 0.01720285415649414), (16, 924, 0.00810694694519043)]
